# Avito Deal Prediction Project

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from scipy.stats import itemfreq
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from skimage import feature
from PIL import Image as IMG
import operator
import cv2
import os 

from IPython.core.display import HTML 
from IPython.display import Image

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

## Model Building

### 1.GLM

In [100]:
h2o.shutdown(prompt=False)

[WARNING] in <ipython-input-100-02ff3806b010> line 1:
    >>> h2o.shutdown(prompt=False)
        ^^^^ Deprecated, use ``h2o.cluster().shutdown()``.
H2O session _sid_9952 closed.


In [60]:
newtrain_train=pd.read_csv('train_new_wanmo.csv')
newtrain_validate=pd.read_csv('validate_new_wanmo.csv')
newtest=pd.read_csv('test_new_wanmo.csv')

In [12]:
newtest.columns

Index(['item_id', 'price', 'user_type', 'image_top_1', 'size', 'width',
       'height', 'brightness', 'average_red', 'average_green', 'average_blue',
       'blurrness', 'region_en', 'parent_category_name_en', 'category_name_en',
       'image_dummy', 'price_dummy', 'day of week', 'month', 'title_num',
       'describe_num', 'seq_group15', 'param_1_english', 'param_2_english',
       'param_3_english'],
      dtype='object')

In [13]:
newtrain_train=newtrain_train.drop(newtrain_train.columns[23:], axis=1)

In [14]:
newtrain_validate=newtrain_validate.drop(newtrain_validate.columns[23:], axis=1)

In [15]:
newtest=newtest.drop(newtest.columns[22:], axis=1)

In [2]:
import time         # for timers
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator 
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.xgboost import H2OXGBoostEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator

In [102]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-win64) (build 25.121-b15, mixed mode)
  Starting server from C:\Users\wchen\AppData\Local\Continuum\anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\wchen\AppData\Local\Temp\tmp7n9dfhch
  JVM stdout: C:\Users\wchen\AppData\Local\Temp\tmp7n9dfhch\h2o_wchen_started_from_python.out
  JVM stderr: C:\Users\wchen\AppData\Local\Temp\tmp7n9dfhch\h2o_wchen_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,10 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.1
H2O cluster version age:,22 days
H2O cluster name:,H2O_from_python_wchen_edtzvv
H2O cluster total nodes:,1
H2O cluster free memory:,1.740 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [103]:
test = "C:/Users/wchen/Downloads/test_df.csv"
test_df = h2o.import_file(path=test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [104]:
train = "C:/Users/wchen/Downloads/train_df.csv"
train_df = h2o.import_file(path=train)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [105]:
valid = "C:/Users/wchen/Downloads/validate_df.csv"
validate_df = h2o.import_file(path=valid)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [41]:
train_df = h2o.H2OFrame(newtrain_train)
validate_df = h2o.H2OFrame(newtrain_validate)
test_df = h2o.H2OFrame(newtest)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [106]:
import re
import time

def gen_submission(preds, test=test_df):

    """ Generates submission file for Kaggle House Prices contest.
    
    :param preds: Column vector of predictions.
    :param test: Test data.
    
    """
    
    # create time stamp
    time_stamp = re.sub('[: ]', '_', time.asctime())

    # create predictions column
    sub = test_df['item_id'].cbind(preds)
    sub.columns = ['item_id', 'deal_probability']
    
    # save file for submission
    sub_fname = '../Downloads/submission_' + str(time_stamp) + '.csv'
    h2o.download_csv(sub, sub_fname)

In [107]:
train_df['image_top_1']=train_df['image_top_1'].asfactor()
train_df['day of week']=train_df['day of week'].asfactor()
train_df['month']=train_df['month'].asfactor()

In [108]:
validate_df['image_top_1']=validate_df['image_top_1'].asfactor()
validate_df['day of week']=validate_df['day of week'].asfactor()
validate_df['month']=validate_df['month'].asfactor()

In [109]:
test_df['image_top_1']=test_df['image_top_1'].asfactor()
test_df['day of week']=test_df['day of week'].asfactor()
test_df['month']=test_df['month'].asfactor()

In [110]:
y = 'deal_probability'
X = [name for name in train_df.columns if name not in [y]]

In [111]:
X.remove('item_id')

In [112]:
X.remove('param_1_english')


In [113]:
X.remove('param_2_english')

In [114]:
X.remove('param_3_english')

In [115]:
print(y)
print(X)

deal_probability
['price', 'user_type', 'image_top_1', 'size', 'width', 'height', 'brightness', 'average_red', 'average_green', 'average_blue', 'blurrness', 'region_en', 'parent_category_name_en', 'category_name_en', 'image_dummy', 'price_dummy', 'day of week', 'month', 'title_num', 'describe_num', 'seq_group15', 'count']


In [37]:
# elastic net regularized regression 
#   - Gaussian family, i.e. squared loss, for linear regression
#   - L1 for variable selection
#   - L2 for handling multicollinearity
#   - IRLS for handling outliers
#   - with lamba parameter tuning for variable selection

# initialize
glm_model1 = H2OGeneralizedLinearEstimator(family='gaussian',
                                         solver='IRLSM',
                                         standardize=True,
                                         lambda_search=True,
                                         nfolds=5,
                                         fold_assignment='Modulo',
                                         keep_cross_validation_predictions=True)

# train 
glm_model1.train(X, y, training_frame=train_df, validation_frame=validate_df)

# print trained model info
glm_model1

# view detailed results at http://host:ip/flow/index.html

glm Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1530279005094_3


ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 0.05377974657726605
RMSE: 0.23190460663226603
MAE: 0.15789858646742716
RMSLE: 0.17300348291796272
R^2: 0.20450700337727123
Mean Residual Deviance: 0.05377974657726605
Null degrees of freedom: 1052395
Residual degrees of freedom: 1052158
Null deviance: 71147.81703820138
Residual deviance: 56597.59017892848
AIC: -88955.95278479448

ModelMetricsRegressionGLM: glm
** Reported on validation data. **

MSE: 0.05395952109298474
RMSE: 0.2322918877037783
MAE: 0.15824896985473505
RMSLE: 0.17332003560157763
R^2: 0.20323187607177773
Mean Residual Deviance: 0.05395952109298474
Null degrees of freedom: 451027
Residual degrees of freedom: 450790
Null deviance: 30544.975257034417
Residual deviance: 24337.25487952672
AIC: -36345.755512958

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.1581442,0.0001633,0.1577060,0.1581326,0.1583067,0.1582287,0.1583467
mean_residual_deviance,0.0539425,0.0001231,0.0536916,0.0538367,0.0541810,0.0539194,0.0540839
mse,0.0539425,0.0001231,0.0536916,0.0538367,0.0541810,0.0539194,0.0540839
null_deviance,14229.614,44.740883,14157.317,14156.285,14294.602,14238.803,14301.067
r2,0.2020890,0.0010544,0.2017493,0.1995269,0.2022060,0.2029591,0.2040038
residual_deviance,11353.779,25.898746,11301.013,11331.497,11403.965,11348.906,11383.518
rmse,0.232255,0.0002650,0.2317145,0.2320274,0.2327682,0.2322056,0.2325594
rmsle,0.1732915,0.0001373,0.1729706,0.1732480,0.1735408,0.1732626,0.1734355


Scoring History: 


,timestamp,duration,iteration,lambda,predictors,deviance_train,deviance_test,deviance_xval,deviance_se
,2018-06-29 09:52:28,0.000 sec,1,.7E-1,1,0.0676056,0.0677230,0.0676058,0.0001503
,2018-06-29 09:52:28,0.120 sec,2,.64E-1,2,0.0674097,0.0675299,0.0676056,0.0001504
,2018-06-29 09:52:28,0.228 sec,3,.58E-1,2,0.0672384,0.0673611,0.0676056,0.0001504
,2018-06-29 09:52:28,0.337 sec,4,.53E-1,3,0.0669406,0.0670638,0.0674849,0.0001532
,2018-06-29 09:52:28,0.447 sec,5,.48E-1,3,0.0664906,0.0666118,0.0673045,0.0001515
---,---,---,---,---,---,---,---,---,---
,2018-06-29 09:52:55,26.825 sec,96,.1E-4,1617,0.0538851,0.0540430,0.0540274,0.0000792
,2018-06-29 09:52:55,27.340 sec,97,.93E-5,1715,0.0538567,0.0540200,0.0540016,0.0000785
,2018-06-29 09:52:56,27.901 sec,98,.84E-5,1812,0.0538064,0.0539801,0.0539986,0.0000869
,2018-06-29 09:52:56,28.483 sec,99,.77E-5,1931,0.0537797,0.0539595,0.0539906,0.0000718



See the whole table with table.as_data_frame()


In [38]:
print(glm_model1.rmse(train=True))

0.23190460663226603


In [39]:
print(glm_model1.rmse(valid=True))

0.2322918877037783


In [40]:
# save the model
model_path = h2o.save_model(model=glm_model1, path= "C:/Users/wchen/Desktop/models", force=True)

In [41]:
glm_model_test1 = glm_model1.predict(test_df)
gen_submission(glm_model_test1)

glm prediction progress: |████████████████████████████████████████████████| 100%


C:\Users\wchen\AppData\Local\Continuum\anaconda3\lib\site-packages\h2o\job.py:69: UserWarning: Test/Validation dataset column 'image_top_1' has levels not trained on: [200, 614]
  warnings.warn(w)
C:\Users\wchen\AppData\Local\Continuum\anaconda3\lib\site-packages\h2o\job.py:69: UserWarning: Test/Validation dataset column 'param_3_english' has levels not trained on: [110, 146, 2018-09-03 00:00:00, 2018-09-05 00:00:00, 2136, 2335, 240, 26th, 27th, 3000 GT, 3153, 353, 40-42 (XS), 402, 42-44 (S), 44-46 (M), 44-46 (S), 452 Loaf of bread, 46-48 (M), 48-50 (XL), 4Runner, 4x4 (Niva), 5 series GT, 50-52 (XL), 50-56 cm (0-2 months), 62-68 cm (2-6 months), 6th, 74-80 cm (7-12 months), 760, 7th, 8, <19, <35, <40, > 50 (XXL), AD, ATS, Accord, Ace, Aerio, Airwave, Altezza, Altima, Antara, Aristo, Atrai, Avante, Avenir, Axela, B5, Biante, Bora, Brevis, C-3d, CL, CLA class, Cami, Capa, Capella, Caprice, Carisma, Cayman, Cebrium (720), Celsior, Chariot, Clarus, Clio RS, Concorde, Cooper S Clubman, Cord

### 2. Random Forest

### 3. ERT

In [32]:
# ERT with random hyperparameter search
# train many different RF models with random hyperparameters
# and select best model based on validation error

# define random grid search parameters
hyper_parameters = {'ntrees':list(range(0, 500, 50)),
                    'max_depth':list(range(0, 20, 2))}

# define search strategy
search_criteria = {'strategy':'RandomDiscrete',
                   'max_models':20,
                   'max_runtime_secs':1800}

ertsearch = H2OGridSearch(model=H2ORandomForestEstimator(nfolds=5,fold_assignment='Modulo',keep_cross_validation_predictions=True,histogram_type='random'),
                        hyper_params=hyper_parameters,
                        search_criteria=search_criteria)

ertsearch.train(x=X,
              y=y,
              training_frame=train_df,
              validation_frame=validate_df,
              stopping_rounds=10)

drf Grid Build progress: |████████████████████████████████████████████████| 100%


In [33]:
#show grid search results
ertsearch.show()

# select best model
ert_model = ertsearch.get_grid()[0]

# print model information
ert_model

    max_depth ntrees  \
0           8    348   
1          18      0   

                                                     model_ids  \
0  Grid_DRF_py_3_sid_b726_model_python_1530125209496_3_model_1   
1  Grid_DRF_py_3_sid_b726_model_python_1530125209496_3_model_0   

     residual_deviance  
0  0.05807494860078983  
1  0.08695061512881257  
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  Grid_DRF_py_3_sid_b726_model_python_1530125209496_3_model_1


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 0.05806658278131296
RMSE: 0.24097008690149274
MAE: 0.16800827967580556
RMSLE: 0.17862223426049073
Mean Residual Deviance: 0.05806658278131296

ModelMetricsRegression: drf
** Reported on validation data. **

MSE: 0.05818575758167663
RMSE: 0.2412172414685083
MAE: 0.16822178111965558
RMSLE: 0.17876275627233545
Mean Residual Deviance: 0.05818575758167663

ModelMetricsRegression: drf
** Reported on cross-validation data. **

MSE: 0.0580749486007898

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.1680350,0.0002943,0.1679868,0.1678641,0.1688066,0.167551,0.1679666
mean_residual_deviance,0.0580749,0.0001799,0.0577981,0.0578047,0.0584377,0.0580543,0.0582799
mse,0.0580749,0.0001799,0.0577981,0.0578047,0.0584377,0.0580543,0.0582799
r2,0.1409674,0.0006880,0.1406965,0.1405283,0.1395276,0.1418377,0.1422470
residual_deviance,0.0580749,0.0001799,0.0577981,0.0578047,0.0584377,0.0580543,0.0582799
rmse,0.2409869,0.0003732,0.2404124,0.2404261,0.2417390,0.2409445,0.2414123
rmsle,0.1786230,0.0002233,0.1783304,0.1783228,0.1791487,0.1785076,0.1788053


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2018-06-27 15:49:29,27 min 49.388 sec,0.0,nan,nan,nan,nan,nan,nan
,2018-06-27 15:49:30,27 min 50.124 sec,1.0,0.2441291,0.1662241,0.0595990,0.2442330,0.1663048,0.0596498
,2018-06-27 15:49:30,27 min 50.904 sec,2.0,0.2447885,0.1672185,0.0599214,0.2425704,0.1663778,0.0588404
,2018-06-27 15:49:31,27 min 51.807 sec,3.0,0.2455031,0.1678492,0.0602718,0.2433647,0.1670266,0.0592264
,2018-06-27 15:49:32,27 min 52.728 sec,4.0,0.2452623,0.1679053,0.0601536,0.2432325,0.1671286,0.0591620
,2018-06-27 15:49:37,27 min 57.162 sec,10.0,0.2426464,0.1671834,0.0588773,0.2416964,0.1669442,0.0584171
,2018-06-27 15:49:45,28 min 5.365 sec,21.0,0.2420182,0.1669846,0.0585728,0.2417308,0.1670246,0.0584338
,2018-06-27 15:50:02,28 min 22.783 sec,45.0,0.2412279,0.1673214,0.0581909,0.2412678,0.1674670,0.0582101
,2018-06-27 15:50:39,28 min 59.211 sec,93.0,0.2410172,0.1677970,0.0580893,0.2411539,0.1679733,0.0581552
,2018-06-27 15:51:53,30 min 13.823 sec,195.0,0.2410715,0.1679342,0.0581155,0.2412559,0.1681238,0.0582044


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
category_name_en,1869954.3750000,1.0,0.4333431
parent_category_name_en,1089132.6250000,0.5824381,0.2523955
image_top_1,687771.3750000,0.3678012,0.1593841
price,207510.2187500,0.1109707,0.0480884
user_type,128342.5156250,0.0686340,0.0297421
---,---,---,---
brightness,4543.5864258,0.0024298,0.0010529
blurrness,708.7791138,0.0003790,0.0001643
day of week,700.2178955,0.0003745,0.0001623
Unnamed: 0,478.8848572,0.0002561,0.0001110



See the whole table with table.as_data_frame()


In [34]:
print(ert_model.rmse(train=True))

0.24097008690149274


In [35]:
print(ert_model.rmse(valid=True))

0.2412172414685083


In [36]:
ert_model_test = ert_model.predict(test_df)
gen_submission(ert_model_test)

drf prediction progress: |████████████████████████████████████████████████| 100%


C:\Users\wchen\AppData\Local\Continuum\anaconda3\lib\site-packages\h2o\job.py:69: UserWarning: Test/Validation dataset column 'image_top_1' has levels not trained on: [200, 614]
  warnings.warn(w)


### 4.GBM

In [116]:
# GBM with random hyperparameter search
# train many different GBM models with random hyperparameters
# and select best model based on validation error

# define random grid search parameters
hyper_parameters = {'ntrees':list(range(0, 500, 50)),
                    'max_depth':list(range(0, 20, 2)),
                    'sample_rate':[s/float(10) for s in range(1, 11)],
                    'col_sample_rate':[s/float(10) for s in range(1, 11)]}

# define search strategy
search_criteria = {'strategy':'RandomDiscrete',
                   'max_models':20,
                   'max_runtime_secs':1800}

# initialize grid search
gsearch = H2OGridSearch(model=H2OGradientBoostingEstimator(nfolds=5,fold_assignment='Modulo',keep_cross_validation_predictions=True),
                        hyper_params=hyper_parameters,
                        search_criteria=search_criteria)

# execute training w/ grid search
gsearch.train(x=X,
              y=y,
              training_frame=train_df,
              validation_frame=validate_df,
              stopping_rounds=10)

# view detailed results at http://localhost:54321/flow/index.html

gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [117]:
# show grid search results
gsearch.show()

# select best model
gbm_model = gsearch.get_grid()[0]

# print model information
gbm_model

    col_sample_rate max_depth ntrees sample_rate  \
0               0.8        10    150         0.3   
1               0.1         8    300         0.1   
2               0.7        12     50         0.1   
3               0.1         4     97         0.4   
4               0.7        16     73         0.2   

                                                      model_ids  \
0  Grid_GBM_py_35_sid_9c4e_model_python_1530288365896_1_model_2   
1  Grid_GBM_py_35_sid_9c4e_model_python_1530288365896_1_model_1   
2  Grid_GBM_py_35_sid_9c4e_model_python_1530288365896_1_model_0   
3  Grid_GBM_py_35_sid_9c4e_model_python_1530288365896_1_model_4   
4  Grid_GBM_py_35_sid_9c4e_model_python_1530288365896_1_model_3   

      residual_deviance  
0  0.052863268951093396  
1    0.0544884285884608  
2   0.05512482528234771  
3   0.05624821972169613  
4   0.05681176023241796  
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  Grid_GBM_py_35_sid_9c4e_model_python_1530288

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.1536820,0.0002042,0.1531930,0.1535169,0.1539702,0.1538497,0.1538804
mean_residual_deviance,0.0528633,0.0001441,0.0525933,0.0526861,0.0531669,0.0529145,0.0529555
mse,0.0528633,0.0001441,0.0525933,0.0526861,0.0531669,0.0529145,0.0529555
r2,0.2180551,0.0009721,0.2180787,0.2166342,0.2171382,0.2178135,0.2206109
residual_deviance,0.0528633,0.0001441,0.0525933,0.0526861,0.0531669,0.0529145,0.0529555
rmse,0.2299197,0.0003133,0.2293322,0.2295346,0.2305795,0.2300316,0.2301206
rmsle,0.1716739,0.0001630,0.1713281,0.1715535,0.1720268,0.1717496,0.1717116


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2018-06-29 12:23:39,17 min 2.099 sec,0.0,0.2600107,0.1901577,0.0676056,0.2602365,0.1903435,0.0677230
,2018-06-29 12:23:39,17 min 2.790 sec,1.0,0.2553550,0.1859863,0.0652062,0.2557195,0.1862878,0.0653925
,2018-06-29 12:23:40,17 min 3.446 sec,2.0,0.2514562,0.1824426,0.0632302,0.2519886,0.1828659,0.0634983
,2018-06-29 12:23:41,17 min 4.141 sec,3.0,0.2483108,0.1795524,0.0616582,0.2489831,0.1800834,0.0619926
,2018-06-29 12:23:41,17 min 4.870 sec,4.0,0.2454126,0.1768468,0.0602274,0.2462770,0.1775097,0.0606523
,2018-06-29 12:23:42,17 min 5.618 sec,5.0,0.2429160,0.1744539,0.0590082,0.2439555,0.1752582,0.0595143
,2018-06-29 12:23:46,17 min 9.775 sec,12.0,0.2335494,0.1638584,0.0545453,0.2358391,0.1654818,0.0556201
,2018-06-29 12:23:51,17 min 14.054 sec,19.0,0.2294574,0.1582342,0.0526507,0.2330194,0.1606950,0.0542980
,2018-06-29 12:23:55,17 min 18.227 sec,26.0,0.2264028,0.1544813,0.0512582,0.2311929,0.1577111,0.0534502
,2018-06-29 12:23:59,17 min 22.476 sec,33.0,0.2243304,0.1521181,0.0503241,0.2303711,0.1561677,0.0530708


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
category_name_en,34495.9960938,1.0,0.3935661
image_top_1,15225.9306641,0.4413825,0.1737132
region_en,9365.2919922,0.2714892,0.1068490
seq_group15,6541.9521484,0.1896438,0.0746374
parent_category_name_en,4623.0590820,0.1340173,0.0527447
---,---,---,---
count,472.9980774,0.0137117,0.0053965
height,385.3219910,0.0111700,0.0043962
price_dummy,169.4272308,0.0049115,0.0019330
image_dummy,3.8380327,0.0001113,0.0000438



See the whole table with table.as_data_frame()


In [118]:
print(gbm_model.rmse(train=True))

0.2129287322009635


In [119]:
print(gbm_model.rmse(valid=True))

0.22932004750338947


In [120]:
gbm_model4=gbm_model

In [121]:
# save the model
model_path = h2o.save_model(model=gbm_model4, path= "C:/Users/wchen/Desktop/models", force=True)

In [123]:
gbm_model_test = gbm_model.predict(test_df)
gen_submission(gbm_model_test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


C:\Users\wchen\AppData\Local\Continuum\anaconda3\lib\site-packages\h2o\job.py:69: UserWarning: Test/Validation dataset column 'image_top_1' has levels not trained on: [200, 614]
  warnings.warn(w)


### 5. XGBoost

In [49]:
# XGBoost with random hyperparameter search
hyper_parameters = {'ntrees':list(range(0, 500, 50)),
                    'max_depth':list(range(0, 20, 2)),
                    'sample_rate':[s/float(10) for s in range(1, 11)],
                    'col_sample_rate':[s/float(10) for s in range(1, 11)]}

# define search strategy
search_criteria = {'strategy':'RandomDiscrete',
                   'max_models':20,
                   'max_runtime_secs':1800}

# initialize grid search
xbgsearch = H2OGridSearch(model=H2OXGBoostEstimator(nfolds=5,fold_assignment='Modulo',keep_cross_validation_predictions=True),
                        hyper_params=hyper_parameters,
                        search_criteria=search_criteria)

# execute training w/ grid search
xbgsearch.train(x=X,
              y=y,
              training_frame=train_df,
              validation_frame=validate_df,
              stopping_rounds=10)

# view detailed results at http://localhost:54321/flow/index.html

H2OResponseError: Server error water.exceptions.H2ONotFoundArgumentException:
  Error: POST /99/Grid/xgboost not found
  Request: POST /99/Grid/xgboost
    data: {'search_criteria': "{'strategy': 'RandomDiscrete', 'max_models': 20, 'max_runtime_secs': 1800}", 'hyper_parameters': "{'ntrees': [0, 50, 100, 150, 200, 250, 300, 350, 400, 450], 'max_depth': [0, 2, 4, 6, 8, 10, 12, 14, 16, 18], 'sample_rate': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'col_sample_rate': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}", 'nfolds': '5', 'fold_assignment': 'Modulo', 'keep_cross_validation_predictions': 'True', 'stopping_rounds': '10', 'training_frame': 'py_14_sid_855a', 'validation_frame': 'py_17_sid_855a', 'response_column': 'deal_probability', 'ignored_columns': '["item_id"]'}


In [ ]:
# show grid search results
xbgsearch.show()

# select best model
xgb_model = xbgsearch.get_grid()[0]

# print model information
xgb_model

In [ ]:
print(xgb_model.rmse(train=True))

In [ ]:
print(xgb_model.rmse(valid=True))

In [ ]:
# save the model
model_path = h2o.save_model(model=xgb_model, path= "C:/Users/wchen/Desktop/models", force=True)

In [ ]:
xgb_model_test = xgb_model.predict(test_df)
gen_submission(xgb_model_test)

# NN

In [89]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

In [125]:
## NN with random hyperparameter search
# train many different NN models with random hyperparameters
# and select best model based on validation error

# define random grid search parameters
hyper_parameters = {'hidden':[[170, 320], [80, 190], [320, 160, 80], [100], [50, 50, 50, 50]],
                    'l1':[s/1e4 for s in range(0, 1000, 100)],
                    'l2':[s/1e5 for s in range(0, 1000, 100)],
                    'input_dropout_ratio':[s/1e2 for s in range(0, 20, 2)]}

# define search strategy
search_criteria = {'strategy':'RandomDiscrete',
                   'max_models':20,
                   'max_runtime_secs':1800}

# initialize grid search
nnsearch = H2OGridSearch(H2ODeepLearningEstimator(nfolds=5,fold_assignment='Modulo',keep_cross_validation_predictions=True),
                        hyper_params=hyper_parameters,
                        search_criteria=search_criteria)

# execute training w/ grid search
nnsearch.train(x=X,
              y=y,
              training_frame=train_df,
              validation_frame=validate_df)


deeplearning Grid Build progress: |███████████████████████████████████████| 100%


In [126]:
nnsearch.show()

# select best model
nn_model = nnsearch.get_grid()[0]

# print model information
nn_model

        hidden input_dropout_ratio   l1     l2  \
0    [80, 190]                0.18  0.0  0.008   

                                                               model_ids  \
0  Grid_DeepLearning_py_35_sid_9c4e_model_python_1530288365896_4_model_0   

      residual_deviance  
0  0.054739520316708624  
Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  Grid_DeepLearning_py_35_sid_9c4e_model_python_1530288365896_4_model_0


ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 0.053774502103700834
RMSE: 0.2318932989624772
MAE: 0.15354234695365512
RMSLE: 0.1719326838563551
Mean Residual Deviance: 0.053774502103700834

ModelMetricsRegression: deeplearning
** Reported on validation data. **

MSE: 0.05447697573953865
RMSE: 0.23340303284134645
MAE: 0.15471240384460602
RMSLE: 0.17297033670264533
Mean Residual Deviance: 0.05447697573953865

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 0.054739520316708624
RMSE: 0.2339

,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,0.1508904,0.0040583,0.1527832,0.1499340,0.1607815,0.1468458,0.1441074
mean_residual_deviance,0.0549017,0.0003338,0.0544598,0.0545042,0.0547281,0.0550782,0.0557382
mse,0.0549017,0.0003338,0.0544598,0.0545042,0.0547281,0.0550782,0.0557382
r2,0.1879132,0.0034658,0.1903282,0.1896023,0.1941501,0.1858302,0.1796552
residual_deviance,0.0549017,0.0003338,0.0544598,0.0545042,0.0547281,0.0550782,0.0557382
rmse,0.2343090,0.0007111,0.2333663,0.2334613,0.2339404,0.2346874,0.2360894
rmsle,0.1739157,0.0005938,0.1738316,0.1724436,0.1747511,0.1738321,0.1747203


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_deviance,training_mae,training_r2,validation_rmse,validation_deviance,validation_mae,validation_r2
,2018-06-29 13:29:31,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan,nan,nan
,2018-06-29 13:29:32,29 min 27.696 sec,4271 obs/sec,0.0034863,1,3669.0,0.2455370,0.0602884,0.1587248,0.0987433,0.2469175,0.0609683,0.1596570,0.0997405
,2018-06-29 13:34:30,34 min 27.882 sec,9533 obs/sec,2.2213340,637,2337723.0,0.2318933,0.0537745,0.1535423,0.1961204,0.2334030,0.0544770,0.1547124,0.1955911


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
category_name_en.Offer services,1.0,1.0,0.0013066
category_name_en.Cars,0.9398981,0.9398981,0.0012280
parent_category_name_en.Services,0.9011490,0.9011490,0.0011774
parent_category_name_en.Personal belongings,0.7965522,0.7965522,0.0010408
category_name_en.Apartments,0.7717899,0.7717899,0.0010084
---,---,---,---
seq_group15.missing(NA),0.0,0.0,0.0
parent_category_name_en.missing(NA),0.0,0.0,0.0
day of week.missing(NA),0.0,0.0,0.0
user_type.missing(NA),0.0,0.0,0.0



See the whole table with table.as_data_frame()


In [127]:
print(nn_model.rmse(train=True))
print(nn_model.rmse(valid=True))

0.2318932989624772
0.23340303284134645


In [128]:
# save the model
model_path = h2o.save_model(model=nn_model, path= "C:/Users/wchen/Desktop/models", force=True)

In [129]:
nn_model_test = nn_model.predict(test_df)
gen_submission(nn_model_test)

deeplearning prediction progress: |███████████████████████████████████████| 100%


C:\Users\wchen\AppData\Local\Continuum\anaconda3\lib\site-packages\h2o\job.py:69: UserWarning: Test/Validation dataset column 'image_top_1' has levels not trained on: [200, 614]
  warnings.warn(w)


### 6. Stacking Ensemble Model

In [132]:
# load the model
GLM = h2o.load_model("C:/Users/wchen/Desktop/models/GLM")
RF = h2o.load_model("C:/Users/wchen/Desktop/models/Grid_DRF")
GBM = h2o.load_model("C:/Users/wchen/Desktop/models/Grid_GBM")

In [ ]:
ensemble = H2OStackedEnsembleEstimator(base_models=[glm_model,rf_model,gbm_model])
ensemble.train(x=X, y=y, training_frame=train_df,validation_frame=validate_df)

In [ ]:
print(ensemble.rmse(train=True))

In [ ]:
print(ensemble.rmse(valid=True))

In [ ]:
ensemble_test1 = ensemble.predict(test_df)
gen_submission(ensemble_test1)

In [133]:
ensemble2 = H2OStackedEnsembleEstimator(base_models=[GLM,RF,GBM])
ensemble2.train(x=X, y=y, training_frame=train_df,validation_frame=validate_df)

stackedensemble Model Build progress: | (failed)


OSError: Job with key $03017f00000132d4ffffffff$_911f5807a58e0d52f5520e0785c84709 failed with an exception: java.lang.NullPointerException
stacktrace: 
java.lang.NullPointerException
	at hex.StackedEnsembleModel.checkAndInheritModelProperties(StackedEnsembleModel.java:313)
	at hex.ensemble.StackedEnsemble$StackedEnsembleDriver.computeImpl(StackedEnsemble.java:234)
	at hex.ModelBuilder$Driver.compute2(ModelBuilder.java:214)
	at water.H2O$H2OCountedCompleter.compute(H2O.java:1260)
	at jsr166y.CountedCompleter.exec(CountedCompleter.java:468)
	at jsr166y.ForkJoinTask.doExec(ForkJoinTask.java:263)
	at jsr166y.ForkJoinPool$WorkQueue.runTask(ForkJoinPool.java:974)
	at jsr166y.ForkJoinPool.runWorker(ForkJoinPool.java:1477)
	at jsr166y.ForkJoinWorkerThread.run(ForkJoinWorkerThread.java:104)


In [ ]:
print(ensemble.rmse(train=True))
print(ensemble.rmse(valid=True))

In [ ]:
ensemble_test1 = ensemble.predict(test_df)
gen_submission(ensemble_test1)